In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from DecisionTree import DecisionTree
# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [35]:
FOLDER = "data/"
FILES = ["iris.txt", "bezdekIris.txt", "wine.txt", "bupa.txt"]
FILE = FILES[2]
df = pd.read_csv(FOLDER+FILE, sep=",", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [41]:
# Подготавливаем признаки и целевую функцию
if FILE in FILES[:2]:
    # ИРИСЫ
    df[4] = pd.factorize(df[4])[0]
    X = df[[0,1,2,3]].as_matrix()
    y = df[4].as_matrix()
    
if FILE in FILES[2]:
    x_indexes = [x for x in range(1,14)]
    X = df[x_indexes].as_matrix()
    y = df[0].as_matrix()

In [43]:
x_train, x_test, y_train, y_test = cv.train_test_split(X, y, test_size=0.25)

In [59]:
%%time
my_tree = DecisionTree()
my_tree.fit(x_train, y_train)
y_predicted = my_tree.predict(x_test)
print accuracy_score(y_test, y_predicted)

0.911111111111
CPU times: user 196 ms, sys: 15 µs, total: 196 ms
Wall time: 196 ms


In [46]:
%%time
sklearn_tree = DecisionTreeClassifier(random_state=True)
sklearn_tree.fit(x_train, y_train)
y_predicted = sklearn_tree.predict(x_test)
print accuracy_score(y_test, y_predicted)

0.866666666667
CPU times: user 3.07 ms, sys: 0 ns, total: 3.07 ms
Wall time: 5.19 ms
